# Customer Churn Prediction (ANN)

## Description

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Importing Required Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf

#### Checking if TensorFlow is using GPU properly

In [19]:
gpus = tf.config.list_physical_devices('GPU') #this is the main code for listing all available GPU(s)
if gpus:
    print(f"✅ TensorFlow detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu}")
else:
    print("❌ No GPU detected. TensorFlow is using CPU instead.")

✅ TensorFlow detected 1 GPU(s):
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 1: Data Preprocessing

In [166]:
df = pd.read_csv('C:\\Users\\Amin\\DeepLearningProjects\\my-codes\\Data\\Churn_Modelling.csv')

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [30]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


It seems that the first three features (columns) are not useful and as a resutl, we will drop those features. Plus, we will split our data into features (X) and targets (y)

In [147]:
X = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
y = df['Exited']

In [149]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [151]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

#### Another Approach

In [153]:
X = df.iloc[:, 3:-1]
y = df.iloc[:, -1]

In [155]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [137]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [139]:
X = X.values
y = y.values

In [141]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [143]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 2: Encoding Categorical Data

Due to the fact that the input of Deep Learning must be numerical, we have to encode the categorical features (like Geography and Gender in this dataset)

#### One-Hot Encoding to the 'Gender' column (using pd.get_dummies())

In [168]:
X = pd.get_dummies(X, columns=['Gender'], drop_first=True)

In [176]:
X['Gender_Male'] = X['Gender_Male'].astype(int)

In [178]:
X.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male
0,619,France,42,2,0.00,1,1,1,101348.88,0
1,608,Spain,41,1,83807.86,1,0,1,112542.58,0
2,502,France,42,8,159660.80,3,1,0,113931.57,0
3,699,France,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,43,2,125510.82,1,1,1,79084.10,0


---

#### One Hot Encoding for 'Geoghraphy' Column (using library)

In [68]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

This approach should be implemented on numpy array because if it applies to the pandas dataframe, the display of numbers will be different.

In [188]:
ct.fit_transform(X)[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       4.2000000e+01, 2.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0134888e+05, 0.0000000e+00])

In [108]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [199]:
X = X.values
y = y.values

In [201]:
# Apply OneHotEncoder to 'Geography' (column index 1)
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

In [203]:
X = ct.fit_transform(X)

In [205]:
X[0]

array([1.0, 0.0, 0.0, 619, 42, 2, 0.0, 1, 1, 1, 101348.88, 0],
      dtype=object)

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 3: Scaling the Features

Deep Learning models require scaled data. Therefore, we need to scale them before training our model

In [213]:
from sklearn.preprocessing import StandardScaler

In [215]:
scaler = StandardScaler()

In [217]:
X[0]

array([1.0, 0.0, 0.0, 619, 42, 2, 0.0, 1, 1, 1, 101348.88, 0],
      dtype=object)

In [225]:
X = scaler.fit_transform(X)

In [227]:
X[0]

array([ 0.99720391, -0.57873591, -0.57380915, -0.32622142,  0.29351742,
       -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
        0.02188649, -1.09598752])

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 4 : Splitting the Data

#### Splitting the Data set into Training set and Test set

In [238]:
from sklearn.model_selection import train_test_split

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [244]:
len(X_train)

6700

In [246]:
len(X_test)

3300

In [248]:
len(y_train)

6700

In [250]:
len(y_test)

3300

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 5: Bulding the ANN

#### Initializing the Ann

In [256]:
ann = tf.keras.models.Sequential()

For now, we are going to use only one hidden layer, so we will be able to compare with a model with higher hidden layers

#### Adding the input Layer and First Hidden Layer

# 🧠 Understanding the First Hidden Layer in an ANN

In a **Sequential Artificial Neural Network (ANN)**, the **first hidden layer also serves as the input layer** because it directly receives the raw input features. Unlike traditional architectures where an explicit input layer is defined, **Keras automatically infers the input shape** when the first hidden layer is added.

## 🔢 Choosing the Number of Neurons
The number of neurons (**units**) in this first hidden layer is a crucial hyperparameter. While there is no fixed rule, a commonly used **rule of thumb** is:

Number of neurons = (number of input features + number of output neurons) / 2

## 🎯 Key Points:
- This **first hidden layer processes the raw inputs** and passes them to the next layer.
- The above rule **provides a balanced starting point** for determining the number of neurons.
- **Experimentation is key!** Try values like **8, 16, 32, or more**, and adjust based on performance.

🔍 *Optimizing this number can help balance learning capacity and computational efficiency!* 🚀


In [268]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

#### Adding the Second Hidden Layer

In [271]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

#### Adding the Output Layer

In [274]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Part6 : Training the ANN

#### Compiling the ANN

In [283]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Training the ANN on Training Set

In [286]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
210/210 [==============================] - 2s 2ms/step - loss: 0.6074 - accuracy: 0.7616
Epoch 2/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4930 - accuracy: 0.7919
Epoch 3/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4601 - accuracy: 0.7919
Epoch 4/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4475 - accuracy: 0.7919
Epoch 5/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4406 - accuracy: 0.7945
Epoch 6/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4357 - accuracy: 0.8072
Epoch 7/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4315 - accuracy: 0.8093
Epoch 8/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4274 - accuracy: 0.8154
Epoch 9/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4229 - accuracy: 0.8200
Epoch 10/100
210/210 [==============================] - 0s 2ms/step - loss: 0.4188 - accura

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Part7 : Making the Predictions and Evaluation the Model

In [311]:
y_pred = ann.predict(X_test)

104/104 [==============================] - 0s 680us/step


In [307]:
y_pred = (y_pred > 0.5).astype(int)

In [309]:
y_pred[:10]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

### Evaluation

In [293]:
# Evaluate the model on test data
test_loss, test_accuracy = ann.evaluate(X_test, y_test)

# Print results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

104/104 [==============================] - 0s 2ms/step - loss: 0.3391 - accuracy: 0.8633
Test Loss: 0.3391
Test Accuracy: 0.8633


#### Confusion Matrix

In [300]:
from sklearn.metrics import confusion_matrix, classification_report

In [298]:
y_pred = (ann.predict(X_test) > 0.5).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

104/104 [==============================] - 0s 864us/step
[[2518  139]
 [ 312  331]]


In [302]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      2657
           1       0.70      0.51      0.59       643

    accuracy                           0.86      3300
   macro avg       0.80      0.73      0.76      3300
weighted avg       0.85      0.86      0.85      3300



<hr style="height:1px;border-width:0;color:black;background-color:black">

### Part 8: Improving the ANN model

In [318]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [320]:
new_ann = Sequential()

In [338]:
# Input & First Hidden Layer (Increased Neurons)
new_ann.add(Dense(units=16, activation='relu'))  

# Second Hidden Layer (More Neurons)
new_ann.add(Dense(units=16, activation='relu'))

# Third Hidden Layer (Extra Layer)
new_ann.add(Dense(units=8, activation='relu'))

In [340]:
# Dropout Layer (Reduces Overfitting)
new_ann.add(Dropout(rate=0.2))  # 20% of neurons will be ignored during training

In [342]:
# Output Layer (Binary Classification)
new_ann.add(Dense(units=1, activation='sigmoid'))

#### Compiling the model

In [344]:
new_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Training the model

In [346]:
new_ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
210/210 [==============================] - 1s 3ms/step - loss: 0.5200 - accuracy: 0.7887
Epoch 2/100
210/210 [==============================] - 1s 3ms/step - loss: 0.4422 - accuracy: 0.8128
Epoch 3/100
210/210 [==============================] - 1s 3ms/step - loss: 0.4119 - accuracy: 0.8239
Epoch 4/100
210/210 [==============================] - 1s 2ms/step - loss: 0.3929 - accuracy: 0.8342
Epoch 5/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3835 - accuracy: 0.8419
Epoch 6/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3714 - accuracy: 0.8433
Epoch 7/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3650 - accuracy: 0.8463
Epoch 8/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3645 - accuracy: 0.8475
Epoch 9/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3617 - accuracy: 0.8496
Epoch 10/100
210/210 [==============================] - 1s 3ms/step - loss: 0.3595 - accura

#### Building an ANN model with more layers and neurons

In [359]:
import time

In [349]:
ann_2 = Sequential()

In [353]:
# First Hidden Layer (More Neurons)
ann_2.add(Dense(units=64, activation='relu'))  

# Second Hidden Layer
ann_2.add(Dense(units=64, activation='relu'))  

# Third Hidden Layer
ann_2.add(Dense(units=32, activation='relu'))

# Dropout (Prevents Overfitting)
ann_2.add(Dropout(rate=0.3))  # 30% dropout

# Fourth Hidden Layer
ann_2.add(Dense(units=16, activation='relu'))

# Output Layer (Binary Classification)
ann_2.add(Dense(units=1, activation='sigmoid'))

In [355]:
ann_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [361]:
train_start = time.time()

# Train the Model
ann_2.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1)

# Measure Training Time End
train_end = time.time()

Epoch 1/150
210/210 [==============================] - 1s 3ms/step - loss: 0.4744 - accuracy: 0.7854
Epoch 2/150
210/210 [==============================] - 1s 3ms/step - loss: 0.4058 - accuracy: 0.8328
Epoch 3/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3703 - accuracy: 0.8454
Epoch 4/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3559 - accuracy: 0.8530
Epoch 5/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3493 - accuracy: 0.8576
Epoch 6/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3478 - accuracy: 0.8590
Epoch 7/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3355 - accuracy: 0.8624
Epoch 8/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3341 - accuracy: 0.8612
Epoch 9/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3283 - accuracy: 0.8654
Epoch 10/150
210/210 [==============================] - 1s 3ms/step - loss: 0.3284 - accura

In [367]:
print(f"\nTotal Training Time: {train_end - train_start:.2f} seconds")


Total Training Time: 93.75 seconds


In [365]:
test_loss2, test_accuracy2 = ann_2.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy2:.4f}")

104/104 [==============================] - 0s 2ms/step - loss: 1.2530 - accuracy: 0.8179
Test Accuracy: 0.8179


In [369]:
y_pred2 = (ann_2.predict(X_test) > 0.5).astype(int)

104/104 [==============================] - 0s 2ms/step


In [373]:
report2 = classification_report(y_test, y_pred2)
print(report2)

              precision    recall  f1-score   support

           0       0.89      0.88      0.89      2657
           1       0.53      0.55      0.54       643

    accuracy                           0.82      3300
   macro avg       0.71      0.72      0.71      3300
weighted avg       0.82      0.82      0.82      3300



---

# The End